# Overview

This notebook imports raw ws3 input data, reformats and monkey-patches the data, and exports Woodstock formatted input data files (which we will use in other DSS notebooks for this case as the input data files). 

# Set up environment

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import ws3.forest, ws3.core
import csv
import numpy as np
import time
from functools import partial, wraps
import distance
import operator
import os

Define some key model parameters (will get used but defined here up top for convenience).

In [3]:
period_length = 10
max_age =  1000

In [4]:
# Input paths
# TSA 03, 12, 14, 20, 24
vdyp_curves_smooth_tsa03_path = './data/vdyp_curves_smooth-tsa03.feather'
vdyp_curves_smooth_tsa12_path = './data/vdyp_curves_smooth-tsa12.feather'
vdyp_curves_smooth_tsa14_path = './data/vdyp_curves_smooth-tsa14.feather'
vdyp_curves_smooth_tsa20_path = './data/vdyp_curves_smooth-tsa20.feather'
vdyp_curves_smooth_tsa24_path = './data/vdyp_curves_smooth-tsa24.feather'

stands_shape_file_path = './data/mining_site_1_large_data/mining_site_1.shp'


# Output paths
woodstock_model_files_lan_path = './data/woodstock_model_files_mining_site_1/mining_site_1.lan'
woodstock_model_files_are_path = './data/woodstock_model_files_mining_site_1/mining_site_1.are'
woodstock_model_files_yld_path = './data/woodstock_model_files_mining_site_1/mining_site_1.yld'
woodstock_model_files_act_path = './data/woodstock_model_files_mining_site_1/mining_site_1.act'
woodstock_model_files_trn_path = './data/woodstock_model_files_mining_site_1/mining_site_1.trn'

In [5]:
yld_vdyp_03 = pd.read_feather(vdyp_curves_smooth_tsa03_path)
yld_vdyp_12 = pd.read_feather(vdyp_curves_smooth_tsa12_path)
yld_vdyp_14 = pd.read_feather(vdyp_curves_smooth_tsa14_path)
yld_vdyp_20 = pd.read_feather(vdyp_curves_smooth_tsa20_path)
yld_vdyp_24 = pd.read_feather(vdyp_curves_smooth_tsa24_path)

# Import and reformat inventory and yield input data

Read forest inventory data into memory (vector polygon GIS data layer with attribute table, in ESRI Shapefile format). This dataset represents timber supply area (TSA) 04 in British Columbia. We monkey-patch the inventory data here to make it line up nicely with what we need downstream as input for the ws3 model (i.e., changes we make here to the in-memory dataset are not saved to the original dataset on disk). Most of what we are doing here is setting up the _theme_ columns in the attribute table, which should help newer ws3 users make the connection between input data and the landscape themes in ws3 model further down.

In [6]:
Start = time.time()
stands = gpd.read_file(stands_shape_file_path)
print('It took', round((time.time() - Start) / 60, 1), "minutes to run this script.")
# stands

It took 1.7 minutes to run this script.


Import CANFI tree species lookup table (associates tree species names with integer numerical values, which we use as theme data values in the ws3 model), and insert species code values into the yield curve dataframe.

Here we are removing the satnds with wrong value in the age column (equals with 42 stands).

In [7]:
stands = stands.loc[stands['Age_2024'] != 2024]
# stands

In [8]:
canfi_map = {'AC':1211, 
             'AT':1201,
             'ACT':1201,
             'BL':304,
             'B' :304, #based on 00_data_prep_Resultant.ipynp
             'BA': 304, # added manually
             'EP':1303, 
             'FDI':500, 
             'H' :400,
             'HW':402,
             'HM':403,
             'PL':204, 
             'PLI':204,
             'PA':204,
             'SB':101, 
             'SE':104, 
             'SW':105, 
             'SX':100,
             'S':100,
             'SXS':100, # added manually
             'SXW':100, # added manually
             'CW': 701, # Eastern white-cedar           
             'AT+SX':1201,
             'SX+AT':100,
             'SX+ACT' :100,
             'AT+PLI' :1201,
             'PLI+AT' : 204,
             'FDI' : 500 # Douglas fir             
            }

In [9]:
Aspen = ['AC', 'ACT', 'AT', 'EP', 'VB', 'MB', 'AT+SX']
Bal = ['B', 'BA', 'BG', 'BL']
Cedar = ['CW', 'YC']
Alder = ['D', 'DR']
DougFir = ['F', 'FD', 'FDC', 'FDI']
Hem = ['H', 'HM', 'HW']
Pine = ['PA', 'PL', 'PLC', 'PW', 'PLI', 'PY']
Spruce = ['S', 'SS', 'SW', 'SX', 'SE', 'SXW', 'SB', 'SXS', 'SX+AT']

Burn CANFI species codes into yield data tables.

In [10]:
def canfi_species(stratum_code):
    s = stratum_code.split('_')[-1].split('+')[0]
    result = canfi_map[s]
    return result

In [11]:
def stratify_stand(r, lexmatch=False, lexmatch_fieldname_suffix='_lexmatch'):
    result = ''
    if lexmatch:
        result += 3 * r['BEC_ZONE_CODE%s' % lexmatch_fieldname_suffix]
        result += '_'
        result += 2 * r['SPECIES_CD_1%s' % lexmatch_fieldname_suffix]
        if r.BCLCS_LEVE == 'TM' and r.SPECIES__1 != None:
            result += '+' + r['SPECIES_CD_2%s' % lexmatch_fieldname_suffix]
    else:
        result += r.BEC_ZONE_C
        result += '_'
        result += r.SPECIES_CD
        if r.BCLCS_LEVE == 'TM' and r.SPECIES__1 != None:
            result += '+' + r.SPECIES__1
    return result

In [12]:
# Check whether'SPECIES_CD' is None

print(stands[stands['SPECIES_CD'].isna()])

       TSA_NUMBER  FEATURE_AR BCLCS_LEVE BEC_ZONE_C  SITE_INDEX SPECIES_CD  \
486            20         0.0         SL        SBS         0.0       None   
535            20         0.0         SL        SBS         0.0       None   
1136           20         0.0         SL        SBS         0.0       None   
1137           20         0.0         SL        SBS         0.0       None   
1195           20         0.0         SL        SBS         0.0       None   
...           ...         ...        ...        ...         ...        ...   
419090         14         0.0         SL       ESSF         0.0       None   
419131         14         0.0         SL        SBS         0.0       None   
419205         14         0.0       None        SBS         0.0       None   
420465         03         0.0         HE       ESSF         0.0       None   
420469         12         0.0         HE        ICH         0.0       None   

        SPECIES_PC SPECIES__1  SPECIES__2   Shape_Leng     Shap

There are no species data for 3,973 stands, so we used the dominant species for each BEC zone in each TSA to fill in the missing information.

| TSA_NUMBER | BEC_ZONE_C    | SPECIES_CD   |
|------------|---------------|--------------|
| 3          | BAFA          | BL           |
| 3          | CWH           | BL           |
| 3          | ESSF          | BL           |
| 3          | ICH           | SX           |
| 3          | SBS           | AT           |
| 12         | BAFA          | BL           |
| 12         | CWH           | HW           |
| 12         | ESSF          | B            |
| 12         | ICH           | AT           |
| 12         | MH            | B            |
| 14         | ESSF          | BL           |
| 14         | SBS           | SX           |
| 20         | BAFA          | BL           |
| 20         | CWH           | HM           |
| 20         | ESSF          | BL           |
| 20         | SBS           | SX           |
| 24         | ESSF          | SE           |
| 24         | SBS         AT           |







In [13]:
species_mapping = {
('03',	'BAFA'): 	'BL',
('03',	'CWH'):	'BL',
('03',	'ESSF'):	'BL',
('03',	'ICH'):	'SX',
('03',	'SBS'):	'AT',
('12',	'BAFA'):	'BL',
('12',	'CWH'):	'HW',
('12',	'ESSF'):	'B',
('12',	'ICH'):	'AT',
('12',	'MH'):	'B',
('14',	'ESSF'):	'BL',
('14',	'SBS'):	'SX',
('20',	'BAFA'):	'BL',
('20',	'CWH'):	'HM',
('20',	'ESSF'):	'BL',
('20',	'SBS'):	'SX',
('24',	'ESSF'):	'SE',
('24',	'SBS'):	'AT'
}

In [14]:
def update_null_species_cd(row):
    if pd.isna(row['SPECIES_CD']):
        key = (row['TSA_NUMBER'], row['BEC_ZONE_C'])
        # print(f"Row: {row}, Key: {key}")  # Debug statement
        return species_mapping.get(key, row['SPECIES_CD'])
    return row['SPECIES_CD']

In [15]:
stands['SPECIES_CD'] = stands.apply(update_null_species_cd, axis=1)

In [16]:
# check if 'SPECIES_CD' is still None
print(stands[stands['SPECIES_CD'].isna()])

Empty GeoDataFrame
Columns: [TSA_NUMBER, FEATURE_AR, BCLCS_LEVE, BEC_ZONE_C, SITE_INDEX, SPECIES_CD, SPECIES_PC, SPECIES__1, SPECIES__2, Shape_Leng, Shape_Area, contclass, rollup, netdown, THLB_Area, Block_ID, Age_2024, geometry]
Index: []


In [17]:
stands['BEC_ZONE_CODE_lexmatch'] = stands.BEC_ZONE_C.str.ljust(4, fillchar='x')

stands['SPECIES_CD_1_lexmatch'] = stands['SPECIES_CD'].str.ljust(4, 'x')
stands['SPECIES_CD_1_lexmatch'] = stands['SPECIES_CD'].str[:1] + stands['SPECIES_CD']

stands['SPECIES_CD_2_lexmatch'] = stands['SPECIES__1'].str.ljust(4, 'x')
stands['SPECIES_CD_2_lexmatch'] = stands['SPECIES__1'].str[:1] + stands['SPECIES__1']

stratify_stand = stratify_stand
stratify_stand_lexmatch = partial(stratify_stand, lexmatch=True)

stands['stratum'] = stands.apply(stratify_stand, axis=1)
stands['stratum_lexmatch'] = stands.apply(stratify_stand_lexmatch, axis=1)

Read yield data from a CSV file and recast AU column data type to integer.

In [18]:
# Define a function to process yld_vdyp for each TSA
def process_yld_vdyp_dataframe(path, start_value):
    df = pd.read_feather(path)
    df['canfi_species'] = df.apply(lambda row: canfi_species(row['stratum_code']), axis=1).astype(int)
    df['au_vdyp'] = pd.factorize(df['stratum_code'] + '_' + df['si_level'])[0] + start_value
    return df

paths = {
    'yld_vdyp_03': (vdyp_curves_smooth_tsa03_path, 1),
    'yld_vdyp_12': (vdyp_curves_smooth_tsa12_path, 46),
    'yld_vdyp_14': (vdyp_curves_smooth_tsa14_path, 91),
    'yld_vdyp_20': (vdyp_curves_smooth_tsa20_path, 181),
    'yld_vdyp_24': (vdyp_curves_smooth_tsa24_path, 225),
}

dataframes = {name: process_yld_vdyp_dataframe(path, start_value) for name, (path, start_value) in paths.items()}

yld_vdyp_03 = dataframes['yld_vdyp_03']
yld_vdyp_12 = dataframes['yld_vdyp_12']
yld_vdyp_14 = dataframes['yld_vdyp_14']
yld_vdyp_20 = dataframes['yld_vdyp_20']
yld_vdyp_24 = dataframes['yld_vdyp_24']

# print(yld_vdyp_03.head())
# print(yld_vdyp_12.head())
# print(yld_vdyp_14.head())
# print(yld_vdyp_20.head())
# print(yld_vdyp_24.head())

In [19]:
# Make some modification on yld_vdyp_12 to remove the stratum codes that are not in the satnds' stratum

yld_vdyp_12= yld_vdyp_12[~yld_vdyp_12['stratum_code'].isin(['SBS_SX', 'SBS_PLI', 'SBS_AT', 'SBS_BL', 'ESSF_SE', 'ESSF_PLI', 'SBS_AT+SX', 'SBS_SX+AT', 'SBS_PL'])]
# yld_vdyp_12

Now we merge all yiled curves for all TSAs into one data frame. 

In [20]:
yld_vdyp_14= yld_vdyp_14[~yld_vdyp_14['stratum_code'].isin(['ICH_HW', 'ICH_AT', 'ICH_SX', 'ICH_BL', 'ESSF_B', 'ICH_PL', 'ICH_H', 'ICH_AC', 'ICH_EP', 'SBS_ACT', 'SBS_SX+ACT', 'ICH_PLI', 'ICH_S'])]
# yld_vdyp_14

In [21]:
yld_vdyp_20= yld_vdyp_20[~yld_vdyp_20['stratum_code'].isin(['ICH_HW', 'ICH_AT', 'ICH_SX', 'ICH_BL'])]
# yld_vdyp_20

In [22]:
yld_vdyp_24= yld_vdyp_24[~yld_vdyp_24['stratum_code'].isin(['ICH_HW'])]
# yld_vdyp_24

Here we need to split the satnds based on the TSA number to reduce the compution time for lexmatch. 

In [23]:
stratum_col = 'stratum'

In [24]:
stands_ = stands.reset_index().set_index(stratum_col)

In [25]:
tsa_numbers = ['03', '12', '14', '20', '24']

stands_dict = {}
for tsa in tsa_numbers:
    stands_dict[f'stands_{tsa}_'] = stands_[stands_['TSA_NUMBER'].isin([tsa])]

stands_03_ = stands_dict['stands_03_']
stands_12_ = stands_dict['stands_12_']
stands_14_ = stands_dict['stands_14_']
stands_20_ = stands_dict['stands_20_']
stands_24_ = stands_dict['stands_24_']


In [26]:
yld_vdyp_dict = {}

for tsa in tsa_numbers:
    df_name = f'yld_vdyp_{tsa}'
    yld_vdyp_dict[f'yld_vdyp_{tsa}_'] = globals()[df_name].reset_index().set_index('stratum_code')

yld_vdyp_03_ = yld_vdyp_dict['yld_vdyp_03_']
yld_vdyp_12_ = yld_vdyp_dict['yld_vdyp_12_']
yld_vdyp_14_ = yld_vdyp_dict['yld_vdyp_14_']
yld_vdyp_20_ = yld_vdyp_dict['yld_vdyp_20_']
yld_vdyp_24_ = yld_vdyp_dict['yld_vdyp_24_']

In [27]:
# Start = time.time()
# names1_03 = set(stands_03_.loc[yld_vdyp_03_.index.values].stratum_lexmatch.unique())
# print('It took', round((time.time() - Start) / 60, 1), "minutes to run this script.")
# names1_03


# It took 37 minutes to run this script. so we keep the copy of output here for later. 

# {'ESSFESSFESSF_BBLBBL',
#  'ESSFESSFESSF_PPLIPPLI',
#  'ESSFESSFESSF_SSESSE',
#  'ESSFESSFESSF_SSXSSX',
#  'ICHxICHxICHx_AATAAT',
#  'ICHxICHxICHx_BBLBBL',
#  'ICHxICHxICHx_HHWHHW',
#  'ICHxICHxICHx_SSXSSX',
#  'SBSxSBSxSBSx_AATAAT',
#  'SBSxSBSxSBSx_AATAAT+SSX',
#  'SBSxSBSxSBSx_BBLBBL',
#  'SBSxSBSxSBSx_PPLIPPLI',
#  'SBSxSBSxSBSx_PPLPPL',
#  'SBSxSBSxSBSx_SSXSSX',
#  'SBSxSBSxSBSx_SSXSSX+AAT'}

In [28]:
# Start = time.time()
# names1_12 = set(stands_12_.loc[yld_vdyp_12_.index.values].stratum_lexmatch.unique())
# print('It took', round((time.time() - Start) / 60, 1), "minutes to run this script.")
# names1_12

# It took 2.5 minutes to run this script.
# {'ESSFESSFESSF_BBLBBL',
#  'ESSFESSFESSF_SSXSSX',
#  'ICHxICHxICHx_AATAAT',
#  'ICHxICHxICHx_BBLBBL',
#  'ICHxICHxICHx_HHWHHW',
#  'ICHxICHxICHx_SSXSSX'}

In [29]:
# Start = time.time()
# names1_14 = set(stands_14_.loc[yld_vdyp_14_.index.values].stratum_lexmatch.unique())
# print('It took', round((time.time() - Start) / 60, 1), "minutes to run this script.")
# names1_14

# It took 127.2 minutes to run this script.
# {'ESSFESSFESSF_BBLBBL',
#  'ESSFESSFESSF_PPLIPPLI',
#  'ESSFESSFESSF_SSESSE',
#  'ESSFESSFESSF_SSXSSX',
#  'SBSxSBSxSBSx_AACAAC',
#  'SBSxSBSxSBSx_AATAAT',
#  'SBSxSBSxSBSx_AATAAT+PPLI',
#  'SBSxSBSxSBSx_AATAAT+SSX',
#  'SBSxSBSxSBSx_BBLBBL',
#  'SBSxSBSxSBSx_FFDIFFDI',
#  'SBSxSBSxSBSx_PPLIPPLI',
#  'SBSxSBSxSBSx_PPLIPPLI+AAT',
#  'SBSxSBSxSBSx_PPLPPL',
#  'SBSxSBSxSBSx_SSBSSB',
#  'SBSxSBSxSBSx_SSESSE',
#  'SBSxSBSxSBSx_SSXSSX',
#  'SBSxSBSxSBSx_SSXSSX+AAT'}

In [30]:
# Start = time.time()
# names1_20 = set(stands_20_.loc[yld_vdyp_20_.index.values].stratum_lexmatch.unique())
# print('It took', round((time.time() - Start) / 60, 1), "minutes to run this script.")
# names1_20

# It took 177.8 minutes to run this script.
# {'ESSFESSFESSF_BBLBBL',
#  'ESSFESSFESSF_PPLIPPLI',
#  'ESSFESSFESSF_SSESSE',
#  'ESSFESSFESSF_SSXSSX',
#  'SBSxSBSxSBSx_AATAAT',
#  'SBSxSBSxSBSx_AATAAT+SSX',
#  'SBSxSBSxSBSx_BBLBBL',
#  'SBSxSBSxSBSx_PPLIPPLI',
#  'SBSxSBSxSBSx_PPLPPL',
#  'SBSxSBSxSBSx_SSXSSX',
#  'SBSxSBSxSBSx_SSXSSX+AAT'}

In [31]:
# Start = time.time()
# names1_24 = set(stands_24_.loc[yld_vdyp_24_.index.values].stratum_lexmatch.unique())
# print('It took', round((time.time() - Start) / 60, 1), "minutes to run this script.")
# names1_24

# It took 0.5 minutes to run this script.
# {'ESSFESSFESSF_BBLBBL',
#  'ESSFESSFESSF_PPLIPPLI',
#  'ESSFESSFESSF_SSESSE',
#  'SBSxSBSxSBSx_AATAAT',
#  'SBSxSBSxSBSx_BBLBBL',
#  'SBSxSBSxSBSx_PPLIPPLI',
#  'SBSxSBSxSBSx_SSXSSX'}

In [32]:
# The values here have been obtained from the previous command cells to reduce runtime for the next runs.

names1_03 = {'ESSFESSFESSF_BBLBBL',
 'ESSFESSFESSF_PPLIPPLI',
 'ESSFESSFESSF_SSESSE',
 'ESSFESSFESSF_SSXSSX',
 'ICHxICHxICHx_AATAAT',
 'ICHxICHxICHx_BBLBBL',
 'ICHxICHxICHx_HHWHHW',
 'ICHxICHxICHx_SSXSSX',
 'SBSxSBSxSBSx_AATAAT',
 'SBSxSBSxSBSx_AATAAT+SSX',
 'SBSxSBSxSBSx_BBLBBL',
 'SBSxSBSxSBSx_PPLIPPLI',
 'SBSxSBSxSBSx_PPLPPL',
 'SBSxSBSxSBSx_SSXSSX',
 'SBSxSBSxSBSx_SSXSSX+AAT'}

names1_12 = {'ESSFESSFESSF_BBLBBL',
 'ESSFESSFESSF_SSXSSX',
 'ICHxICHxICHx_AATAAT',
 'ICHxICHxICHx_BBLBBL',
 'ICHxICHxICHx_HHWHHW',
 'ICHxICHxICHx_SSXSSX'}

names1_14 = {'ESSFESSFESSF_BBLBBL',
 'ESSFESSFESSF_PPLIPPLI',
 'ESSFESSFESSF_SSESSE',
 'ESSFESSFESSF_SSXSSX',
 'SBSxSBSxSBSx_AACAAC',
 'SBSxSBSxSBSx_AATAAT',
 'SBSxSBSxSBSx_AATAAT+PPLI',
 'SBSxSBSxSBSx_AATAAT+SSX',
 'SBSxSBSxSBSx_BBLBBL',
 'SBSxSBSxSBSx_FFDIFFDI',
 'SBSxSBSxSBSx_PPLIPPLI',
 'SBSxSBSxSBSx_PPLIPPLI+AAT',
 'SBSxSBSxSBSx_PPLPPL',
 'SBSxSBSxSBSx_SSBSSB',
 'SBSxSBSxSBSx_SSESSE',
 'SBSxSBSxSBSx_SSXSSX',
 'SBSxSBSxSBSx_SSXSSX+AAT'}

names1_20 = {'ESSFESSFESSF_BBLBBL',
 'ESSFESSFESSF_PPLIPPLI',
 'ESSFESSFESSF_SSESSE',
 'ESSFESSFESSF_SSXSSX',
 'SBSxSBSxSBSx_AATAAT',
 'SBSxSBSxSBSx_AATAAT+SSX',
 'SBSxSBSxSBSx_BBLBBL',
 'SBSxSBSxSBSx_PPLIPPLI',
 'SBSxSBSxSBSx_PPLPPL',
 'SBSxSBSxSBSx_SSXSSX',
 'SBSxSBSxSBSx_SSXSSX+AAT'}

names1_24 = {'ESSFESSFESSF_BBLBBL',
 'ESSFESSFESSF_PPLIPPLI',
 'ESSFESSFESSF_SSESSE',
 'SBSxSBSxSBSx_AATAAT',
 'SBSxSBSxSBSx_BBLBBL',
 'SBSxSBSxSBSx_PPLIPPLI',
 'SBSxSBSxSBSx_SSXSSX'}

names1 = names1_03.union(names1_12, names1_14, names1_20, names1_24)

names1

{'ESSFESSFESSF_BBLBBL',
 'ESSFESSFESSF_PPLIPPLI',
 'ESSFESSFESSF_SSESSE',
 'ESSFESSFESSF_SSXSSX',
 'ICHxICHxICHx_AATAAT',
 'ICHxICHxICHx_BBLBBL',
 'ICHxICHxICHx_HHWHHW',
 'ICHxICHxICHx_SSXSSX',
 'SBSxSBSxSBSx_AACAAC',
 'SBSxSBSxSBSx_AATAAT',
 'SBSxSBSxSBSx_AATAAT+PPLI',
 'SBSxSBSxSBSx_AATAAT+SSX',
 'SBSxSBSxSBSx_BBLBBL',
 'SBSxSBSxSBSx_FFDIFFDI',
 'SBSxSBSxSBSx_PPLIPPLI',
 'SBSxSBSxSBSx_PPLIPPLI+AAT',
 'SBSxSBSxSBSx_PPLPPL',
 'SBSxSBSxSBSx_SSBSSB',
 'SBSxSBSxSBSx_SSESSE',
 'SBSxSBSxSBSx_SSXSSX',
 'SBSxSBSxSBSx_SSXSSX+AAT'}

In [33]:
totalarea = stands_.FEATURE_AR.sum()
stands_['totalarea_p'] = stands_.FEATURE_AR / totalarea
names2 = set(stands_.stratum_lexmatch.unique()) - names1
stratum_key = stands_.reset_index().groupby('%s_lexmatch' % stratum_col)[stratum_col].first()
totalarea_p_sum__ = stands_.groupby('%s_lexmatch' % stratum_col).totalarea_p.sum()
lev_dist = {n2:{n1:distance.levenshtein(n1, n2) for n1 in names1} for n2 in names2} 
lev_dist_low = {n2:{n1:(lev_dist[n2][n1], totalarea_p_sum__.loc[n1]) 
                    for n1 in lev_dist[n2].keys() if lev_dist[n2][n1] == min(lev_dist[n2].values())} 
                for n2 in names2}
best_match = {stratum_key.loc[n2]:stratum_key[max(lev_dist_low[n2].items(), key=operator.itemgetter(1))[0]] for n2 in names2}
stands_.reset_index(inplace=True)

In [34]:
# Concatenate the DataFrames vertically
yld_vdyp_ = pd.concat([yld_vdyp_03_, yld_vdyp_12_, yld_vdyp_14_, yld_vdyp_20_, yld_vdyp_24_], axis=0)
yld_vdyp_

,level_0,index,age,volume,si_level,canfi_species,au_vdyp
stratum_code,,,,,,,
SBS_SX,0,8,9,1.062753e-13,L,100,1
SBS_SX,1,9,10,4.118229e-10,L,100,1
SBS_SX,2,10,11,1.858092e-08,L,100,1
SBS_SX,3,11,12,2.237515e-07,L,100,1
SBS_SX,4,12,13,1.413441e-06,L,100,1
...,...,...,...,...,...,...,...
ESSF_PLI,7029,294,295,2.974311e+02,H,204,248
ESSF_PLI,7030,295,296,2.965865e+02,H,204,248
ESSF_PLI,7031,296,297,2.957375e+02,H,204,248


In [35]:
Start = time.time()
def match_stratum(r):
    return r[stratum_col] if r[stratum_col] in yld_vdyp_.index.values else best_match[r[stratum_col]]

stands_['%s_matched' % stratum_col] = stands_.apply(match_stratum, axis=1)
print('It took', round((time.time() - Start) / 60, 1), "minutes to run this script.")

It took 4.7 minutes to run this script.


In [36]:
stands_ = stands_.copy()

In [37]:
stratum_col = '%s_matched' % stratum_col

In [38]:
stands__ = stands_.set_index(stratum_col)

Compile site index (SI) stats, including quantile levels.

In [39]:
stratum_si_stats = stands__.groupby(stratum_col).SITE_INDEX.describe(percentiles=[0, 0.05, 0.20, 0.35, 0.5, 0.65, 0.80, 0.95, 1])

In [40]:
Start = time.time()

# Function to find the closest column in stratum_si_stats and determine the level
def find_closest_column(row):
    si_levelquants = {'L': [0, 5, 20, 35], 'M': [35, 50, 65], 'H': [65, 80, 95, 100]}
    stratum_matched = row['stratum_matched']
    site_index = row['SITE_INDEX']   
    abs_diff = abs(site_index - stratum_si_stats.loc[stratum_matched, ['0%', '5%', '20%', '35%', '50%', '65%', '80%', '95%', '100%']])    
    closest_column = abs_diff.idxmin()    
    for level, quants in si_levelquants.items():
        if int(closest_column[:-1]) in quants:
            return level

stands_['si_level'] = stands_.apply(find_closest_column, axis=1)

print('It took', round((time.time() - Start) / 60, 1), "minutes to run this script.")

It took 2.3 minutes to run this script.


In [41]:
yld_vdyp = pd.concat([yld_vdyp_03, yld_vdyp_12, yld_vdyp_14, yld_vdyp_20, yld_vdyp_24], axis=0)
# yld_vdyp

In [42]:
Start = time.time()

au_vdyp_values = []
for index, row in stands_.iterrows():
    filtered_df = yld_vdyp[(yld_vdyp['stratum_code'] == row['stratum_matched']) & (yld_vdyp['si_level'] == row['si_level'])]    
    if not filtered_df.empty:
        # Get the value of au_vdyp from the first DataFrame and append it to the list
        au_vdyp_values.append(filtered_df['au_vdyp'].values[0])
    else:
        # If no matching rows found, append NaN
        au_vdyp_values.append(None)

# Add the new column 'au_vdyp' to the second DataFrame with the extracted values
stands_['au_vdyp'] = au_vdyp_values

print('It took', round((time.time() - Start) / 60, 1), "minutes to run this script.")

In [76]:
stands_[['BEC_zone_matched', 'species_matched']] = stands_['stratum_matched'].str.split('_', expand=True)

In [ ]:
columns_to_keep = ['TSA_NUMBER', 'contclass', 'Age_2024', 'geometry', 'species_matched', 'au_vdyp']
stands_mdf = stands_[columns_to_keep].copy()
stands_mdf.loc[:,'area'] = stands_mdf.geometry.area * 0.0001 # monkey-patch broken area attribute
stands_mdf =  stands_mdf.rename(columns={'TSA_NUMBER': 'theme0', 'contclass':'theme1', 'Age_2024':'age', 'species_matched':'species', 'au_vdyp':'theme5'})
stands_mdf['theme0'] = stands_mdf['theme0'].replace({'03': 'tsa03', '12': 'tsa12', '14': 'tsa14', '20': 'tsa20', '24': 'tsa24'})
stands_mdf['theme1'] = stands_mdf['theme1'].replace({'C': 1, 'N': 0})
stands_mdf['theme2'] = stands_mdf['theme5']
stands_mdf = stands_mdf.drop(columns='geometry')
stands_mdf.insert(4, 'theme3',  stands_mdf['species'].map(canfi_map)) #Burn CANFI species codes into stand data
stands_mdf['theme3'] = stands_mdf['theme3'].astype(int)
stands_mdf.drop(columns=['species'], inplace=True)
stands_mdf.insert(5, 'theme4', stands_mdf['theme5']) # to be filled out with the scpecies code
stands_mdf.insert(6, 'age', stands_mdf.pop('age'))
stands_mdf = stands_mdf.loc[:, ['theme0', 'theme1', 'theme2', 'theme3', 'theme4', 'theme5', 'area', 'age']]
stands_mdf

In [ ]:
print('whole area (ha) is:', stands_mdf['area'].sum())
print('whole contributing area (ha) is:', stands_mdf[stands_mdf['theme1'] == 1]['area'].sum())
print('whole non-contributing area (ha) is:', stands_mdf[stands_mdf['theme1'] == 0]['area'].sum())

Create analysis unit (AU) dataframe from stands dataframe data.

In [87]:
AU = pd.DataFrame(stands_mdf['theme5']).drop_duplicates()
AU.rename(columns={'theme5':'au_vdyp'}, inplace=True)

Join `AU` and `yld_vdyp` dataframes.

In [88]:
yldmerged = pd.merge(AU, yld_vdyp, on=['au_vdyp'], how='inner')

In [89]:
au_yldmerged = yldmerged['au_vdyp'].drop_duplicates()

Add a new `curve_id` colume that has same data values as `AU` column.

In [90]:
yldmerged['curve_id'] = yldmerged['au_vdyp'] 

Rename stuff to match variable names we expect further down.

In [92]:
stands_table = stands_mdf
curve_points_table = yldmerged
curve_points_table.set_index('au_vdyp', inplace=True)

# Export Woodstock-formatted input files 

We can use the new ws3 model instance we just built to export ws3 input files in Woodstock file format. We do this for three reasons. 

The first reason is that it will be simpler and more compact in the actual DSS notebook to instantiate the `ForestModel` object from these Woodstock-formatted files (and also this will provide an opportunity to demonstrate the existance and usage of the Woodstock model import functions that are built into ws3). 

The second reason is that the process of exporting data from a live `ws3.forest.ForestModel` instance to Woodstock-formatted input data files provides some insight into the internal structure and workings of ws3 models (which can be a challenging thing to get started with, particularly if you do not have a lot of experience building and running forest estate models). 

The third reason is that Woodstock file format is designed to be "human readable" (sort of... nobody ever said it would be super easy or super fun). Picking through the exported Woodstock-formatted files might help some people better understand the structure and details of the model we have built. If you have no experience reading Woodstock-formatted model input data files, then this is going to be trickier (unless you pause here and go take an introductory Woodstock training course of sort). Many forest professionals already have familiarity with Woodstock software and its special file format (through having been exposed to this at some point in their career). 

Start by creating a new subdirectory to hold the new Woodstock-formatted data files.

In [ ]:
!mkdir data/woodstock_model_files_mining_site_1

## LANDSCAPE section

The LANDSCAPE section defines stratification variables (themes) and stratification variable values (basecodes). 

In [94]:
theme_cols=['theme0', # TSA 
            'theme1', # THLB
            'theme2', # TIPSY AUs
            'theme3', # leading species code
            'theme4',  # yield curve ID
            'theme5' # VDYP Aus
           ]
basecodes = [list(map(lambda x: str(x), stands_table[tc].unique())) for tc in theme_cols]
basecodes[2] = list(set(basecodes[2] + list(stands_table['theme2'].astype(str))))
basecodes[3] = list(set(basecodes[3] + list(stands_table['theme3'].astype(str))))
basecodes[4] = list(set(basecodes[4] + list(stands_table['theme4'].astype(str))))
basecodes[5] = list(set(basecodes[5] + list(stands_table['theme5'].astype(str))))

In [95]:
with open(woodstock_model_files_lan_path, 'w') as file:
    print('*THEME Timber Supply Area (TSA)', file=file)
    tsas = ['tsa03', 'tsa12', 'tsa14', 'tsa20', 'tsa24']
    for tsa in tsas:
        print(tsa, file=file)
    print('*THEME Timber Harvesting Land Base (THLB)', file=file)
    for basecode in basecodes[1]: print(basecode, file=file)
    print('*THEME Analysis Unit (AU_TIPSY)', file=file)
    for basecode in basecodes[2]: print(basecode, file=file)
    print('*THEME Leading tree species (CANFI species code)', file=file)
    for basecode in basecodes[3]: print(basecode, file=file)
    print('*THEME Yield curve ID', file=file)
    for basecode in basecodes[4]: print(basecode, file=file)
    print('*THEME Analysis Unit (AU_VDYP)', file=file)
    for basecode in basecodes[5]: print(basecode, file=file)

## AREAS section

The AREAS section defines the initial forest inventory, in terms of how many hectares of which age class are present in which development type (where a development type is defined as a unique sequence of landscape theme variable values).

In [96]:
gstands = stands_table.groupby(theme_cols+['age'])

In [97]:
with open(woodstock_model_files_are_path, 'w') as file:
    for name, group in gstands:
        dtk, age, area = tuple(map(lambda x: str(x), name[:-1])), int(name[-1]), group['area'].sum()
        print('*A', ' '.join(v for v in dtk), age, area, file=file)

## YIELDS section

The YIELDS section defines yield curves (in this example we only track merchantable log volume, but we can use yield curves to track all sorts of other stuff). 

In [98]:
with open(woodstock_model_files_yld_path, 'w') as file:
    tot=[]
    swd=[]
    hwd=[]
    unique_au_rows = curve_points_table[~curve_points_table.index.duplicated(keep='first')]    
    for AU, au_row in unique_au_rows.iterrows():
        yname = 's%04d' % int(au_row.canfi_species)    
        curve_id = au_row.curve_id
        mask = ('?', '?', '?', '?', str(curve_id), str(AU) )
        points = [(r.age, r.volume) for _, r in curve_points_table.loc[AU].iterrows() if not r.age % period_length and r.age <= max_age]
        c = ws3.core.Curve(yname, points=points, type='a', is_volume=True, xmax=max_age, period_length=period_length)
        print('*Y', ' '.join(v for v in mask), file=file)
        print(yname, '1', ' '.join(str(int(c[x])) for x in range(0, 300, 10)), file=file)
        if yname not in tot:
            tot.append(yname)
        if int(au_row.canfi_species) > 1200:
            if yname not in hwd: hwd.append(yname)
        else:
            if yname not in swd: swd.append(yname)
    print('*YC ? ? ? ? ? ?', file=file)
    print('totvol _SUM(%s)' % ', '.join(map(str, tot)), file=file)
    print('swdvol _SUM(%s)' % ', '.join(map(str, swd)), file=file)
    print('hwdvol _SUM(%s)' % ', '.join(map(str, hwd)), file=file)

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(8, 12), sharex=True)

cvol = c
ccai = c.cai()
cmai = c.mai()
cmaiytp = c.mai().ytp()
x_cmai = cmaiytp.lookup(0) # optimal rotation age (i.e., culmination of MAI curve)
labels = 'total volume', 'MAI (and CAI)', 'MAI YTP'

ax[0].plot(*zip(*c.points()))
ax[0].plot([0, x_cmai], [0., cvol[x_cmai]], linestyle='--', color='green')

ax[1].plot(*zip(*c.mai().points()))
ax[1].plot(*zip(*c.cai().points()), linestyle=':')

ax[2].plot(*zip(*c.mai().ytp().points()))
ax[2].axhline(0, color='black')

for i in range(len(ax)):
    ax[i].set_ylabel(labels[i])
    ax[i].set_ylim(0, None)
    ax[i].axvline(x_cmai, color='red')
plt.xlim(0, 300)

## ACTIONS section

The ACTIONS section defines actions that can be applied in the model (e.g., harvesting, planting, thinning, fertilization, etc). 

In [100]:
with open(woodstock_model_files_act_path, 'w') as file:
    print('ACTIONS', file=file)
    print('*ACTION harvest Y', file=file)
    print('*OPERABLE harvest', file=file)
    print('? 1 ? ? ? ? _AGE >= 100 AND _AGE <= 600', file=file)

## TRANSITIONS section

The TRANSITIONS section defines transitions (i.e., transition to a new development type and age class induced by applying a specific action to a specific combination of development type and age class). If there were no transitions in a forest estate model, it would simply be aging (i.e., growing) the forest forward from time step 1 through to time step N.

In [101]:
with open(woodstock_model_files_trn_path, 'w') as file:
    acode = 'harvest'
    print('*CASE', acode, file=file)
    record_au = set()
    for au_id, au_row in stands_table.iterrows():
        if au_row.theme5 in record_au: continue
        if not au_row.theme1: continue
        target_curve_id = au_row.theme4  
        smask = ' '.join(('?', '?', '?', '?', '?', str(target_curve_id)))
        tmask = ' '.join(('?', '?' , '?', '?', str(target_curve_id), '?'))
        print('*SOURCE', smask, file=file)
        print('*TARGET', tmask, '100', file=file)
        record_au.add(au_row.theme5)